In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/ProsusAI/finbert

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/ProsusAI/finbert)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [6]:
import pandas as pd
import torch
# 2. Example news articles mapped to tickers
news_data = [
    {"ticker": "AAPL", "text": "Apple beats Wall Street expectations with record iPhone sales."},
    {"ticker": "AAPL", "text": "Apple faces antitrust lawsuit over App Store practices."},
    {"ticker": "MSFT", "text": "Microsoft announces strong growth in Azure cloud services."},
]

# 3. Function to get sentiment score
def get_sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1).numpy()[0]
    # FinBERT labels: [0: positive, 1: negative, 2: neutral]
    score = probs[0] - probs[1]  # positive - negative
    return score

# 4. Process news
df = pd.DataFrame(news_data)
df["sentiment_score"] = df["text"].apply(get_sentiment_score)

# 5. Aggregate per ticker
agg_scores = df.groupby("ticker")["sentiment_score"].mean().reset_index()

print("Individual news sentiment:")
print(df[["ticker", "text", "sentiment_score"]])

print("\nAggregated sentiment per ticker:")
print(agg_scores)

Individual news sentiment:
  ticker                                               text  sentiment_score
0   AAPL  Apple beats Wall Street expectations with reco...         0.910205
1   AAPL  Apple faces antitrust lawsuit over App Store p...        -0.937437
2   MSFT  Microsoft announces strong growth in Azure clo...         0.907952

Aggregated sentiment per ticker:
  ticker  sentiment_score
0   AAPL        -0.013616
1   MSFT         0.907952


## Remote Inference via Inference Providers
Ensure you have a valid **HF_TOKEN** set in your environment. You can get your token from [your settings page](https://huggingface.co/settings/tokens). Note: running this may incur charges above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, automatically selecting an available inference provider for you.
For more information on how to use the Inference Providers, please refer to our [documentation and guides](https://huggingface.co/docs/inference-providers/en/index).

In [ ]:
import os
os.environ['HF_TOKEN'] = 'YOUR_TOKEN_HERE'

In [ ]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="auto",
    api_key=os.environ["HF_TOKEN"],
)

result = client.text_classification(
    "I like you. I love you",
    model="ProsusAI/finbert",
)